# FLOPS
ref: <https://stackoverflow.com/questions/45085938/tensorflow-is-there-a-way-to-measure-flops-for-a-model>
```python
g = tf.Graph()
run_meta = tf.RunMetadata()
with g.as_default():
    A = tf.Variable(tf.random_normal( [25,16] ))
    B = tf.Variable(tf.random_normal( [16,9] ))
    C = tf.matmul(A,B) # shape=[25,9]

    opts = tf.profiler.ProfileOptionBuilder.float_operation()    
    flops = tf.profiler.profile(g, run_meta=run_meta, cmd='op', options=opts)
    if flops is not None:
        print('Flops should be ~',2*25*16*9)
        print('25 x 25 x 9 would be',2*25*25*9) # ignores internal dim, repeats first
        print('TF stats gives',flops.total_float_ops)
```

In [1]:
import tensorflow as tf
g = tf.Graph()
run_meta = tf.RunMetadata()

In [2]:
with g.as_default():
    A = tf.Variable(tf.random_normal( [25,16] ))
    B = tf.Variable(tf.random_normal( [16,9] ))
    C = tf.matmul(A,B) # shape=[25,9]

    opts = tf.profiler.ProfileOptionBuilder.float_operation()    
    flops = tf.profiler.profile(g, run_meta=run_meta, cmd='op', options=opts)
    if flops is not None:
        print('Flops should be ~',2*25*16*9)
        print('25 x 25 x 9 would be',2*25*25*9) # ignores internal dim, repeats first
        print('TF stats gives',flops.total_float_ops)

Flops should be ~ 7200
25 x 25 x 9 would be 11250
TF stats gives 8288


The different between 7200 & 8288 is caused by `random_normal`.

In [3]:
# ***** (1) Create Graph *****
g = tf.Graph()
sess = tf.Session(graph=g)
with g.as_default():
    A = tf.Variable(initial_value=tf.random_normal([25, 16]))
    B = tf.Variable(initial_value=tf.random_normal([16, 9]))
    C = tf.matmul(A, B, name='output')
    sess.run(tf.global_variables_initializer())
    flops = tf.profiler.profile(g, options = tf.profiler.ProfileOptionBuilder.float_operation())
    print('FLOP before freezing', flops.total_float_ops)
# *****************************        

# ***** (2) freeze graph *****
output_graph_def = graph_util.convert_variables_to_constants(sess, g.as_graph_def(), ['output'])

with tf.gfile.GFile('graph.pb', "wb") as f:
    f.write(output_graph_def.SerializeToString())
# *****************************


# ***** (3) Load frozen graph *****
g2 = load_pb('./graph.pb')
with g2.as_default():
    flops = tf.profiler.profile(g2, options = tf.profiler.ProfileOptionBuilder.float_operation())
    print('FLOP after freezing', flops.total_float_ops)

FLOP before freezing 8288
INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.
FLOP after freezing 7200


### Test flops

In [1]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

In [2]:
def load_pb(pb):
    with tf.gfile.GFile(pb, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

In [3]:
# ***** (1) Create Graph *****
g = tf.Graph()
sess = tf.Session(graph=g)
with g.as_default():
    A = tf.Variable(initial_value=tf.random_normal([25, 16]))
    B = tf.Variable(initial_value=tf.random_normal([16, 9]))
    C = tf.matmul(A, B, name='output')
    sess.run(tf.global_variables_initializer())
    flops = tf.profiler.profile(g, options = tf.profiler.ProfileOptionBuilder.float_operation())
    print('FLOP before freezing', flops.total_float_ops)
# *****************************    

FLOP before freezing 8288


In [11]:
def calculate_flops(model, sess):
    output_graph_def = graph_util.convert_variables_to_constants(sess, g.as_graph_def(), ['output'])
    try:
        os.mkdir('./tmp')
    except:
        pass
        # print("Use the ./tmp dir as model frozen buffer.")
    with tf.gfile.GFile('./tmp/graph.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())
    g2 = load_pb('./tmp/graph.pb')
    with g2.as_default():
        flops = tf.profiler.profile(g2, options = tf.profiler.ProfileOptionBuilder.float_operation())
    # return flops.total_float_ops
    return flops 

In [12]:
flops = calculate_flops(g, sess)

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


In [16]:
flops

name: "_TFProfRoot"
children {
  name: "output"
  float_ops: 7200
  total_float_ops: 7200
  input_shapes {
    value {
      dim {
        size: 25
      }
      dim {
        size: 16
      }
    }
  }
  input_shapes {
    key: 1
    value {
      dim {
        size: 16
      }
      dim {
        size: 9
      }
    }
  }
  total_definition_count: 1
}
total_float_ops: 7200
total_definition_count: 2

In [18]:
print('FLOP before freezing', flops.total_float_ops)

FLOP before freezing 7200


# Size
ref: <https://www.tensorflow.org/mobile/optimizing#model_size>
```sh
docker pull tagineerdai/tensorflow-bazel-build:latest
docker run -it -v /abspath/to/your/tmp/graph.pb:/home/app/graph.pb tagineerdai/tensorflow-bazel-build
cd /home/app/tensorflow
bazel build tensorflow/tools/graph_transforms:summarize_graph
bazel-bin/tensorflow/tools/graph_transforms/summarize_graph --in_graph=./../graph.pb
``` 

Output
```sh
No inputs spotted.
No variables spotted.
Found 1 possible outputs: (name=output, op=MatMul)
Found 544 (544) const parameters, 0 (0) variable parameters, and 0 control_edges
Op types used: 2 Const, 2 Identity, 1 MatMul
To use with tensorflow/tools/benchmark:benchmark_model try these arguments:
bazel run tensorflow/tools/benchmark:benchmark_model -- --graph=graph.pb --show_flops --input_layer= --input_layer_type= --input_layer_shape= --output_layer=output
```

In [2]:
summarize='''No variables spotted.
Found 1 possible outputs: (name=output, op=MatMul)
Found 544 (544) const parameters, 0 (0) variable parameters, and 0 control_edges
Op types used: 2 Const, 2 Identity, 1 MatMul
To use with tensorflow/tools/benchmark:benchmark_model try these arguments:
bazel run tensorflow/tools/benchmark:benchmark_model -- --graph=graph.pb --show_flops --input_layer= --input_layer_type= --input_layer_shape= --output_layer=output'''

In [3]:
summary = summarize.strip().split(' ')
summary

['No',
 'variables',
 'spotted.\nFound',
 '1',
 'possible',
 'outputs:',
 '(name=output,',
 'op=MatMul)\nFound',
 '544',
 '(544)',
 'const',
 'parameters,',
 '0',
 '(0)',
 'variable',
 'parameters,',
 'and',
 '0',
 'control_edges\nOp',
 'types',
 'used:',
 '2',
 'Const,',
 '2',
 'Identity,',
 '1',
 'MatMul\nTo',
 'use',
 'with',
 'tensorflow/tools/benchmark:benchmark_model',
 'try',
 'these',
 'arguments:\nbazel',
 'run',
 'tensorflow/tools/benchmark:benchmark_model',
 '--',
 '--graph=graph.pb',
 '--show_flops',
 '--input_layer=',
 '--input_layer_type=',
 '--input_layer_shape=',
 '--output_layer=output']

In [15]:
size = 0
summary = summarize.strip().split(' ')
while summary[summary.index('const') + 1] != 'parameters,':
    starter = summary.index('const')+1
    summary = summary[starter:]
if len(summary) is not 0:
    size = int(summary[summary.index('const') - 2])

In [16]:
size

544

In [18]:
import os, sys
os.listdir('./tmp')

['MS_code_base', 'libtensorflow_framework.so', 'summarize_graph', 'graph.pb']

In [25]:
import sys
sys.path.insert(0, os.path.abspath('./tmp'))
print(sys.path)

['/home/t-yada/Projects/demo.tf/ModelBuilder/tmp', '', '/usr/lib/python35.zip', '/usr/lib/python3.5', '/usr/lib/python3.5/plat-x86_64-linux-gnu', '/usr/lib/python3.5/lib-dynload', '/home/t-yada/.local/lib/python3.5/site-packages', '/usr/local/lib/python3.5/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.5/dist-packages/IPython/extensions', '/home/t-yada/.ipython', './tmp', './tmp', '/home/t-yada/Projects/demo.tf/ModelBuilder/tmp']


In [27]:
from subprocess import Popen, PIPE

p = Popen(['./tmp/summarize_graph', '--in_graph=./tmp/graph.pb'], stdout=PIPE, stderr=PIPE, stdin=PIPE)

output = p.stdout.read()
output

b'No inputs spotted.\nNo variables spotted.\nFound 1 possible outputs: (name=output, op=MatMul) \nFound 544 (544) const parameters, 0 (0) variable parameters, and 0 control_edges\nOp types used: 2 Const, 2 Identity, 1 MatMul\nTo use with tensorflow/tools/benchmark:benchmark_model try these arguments:\nbazel run tensorflow/tools/benchmark:benchmark_model -- --graph=./tmp/graph.pb --show_flops --input_layer= --input_layer_type= --input_layer_shape= --output_layer=output\n'

In [29]:
import sys
sys.path.insert(0, os.path.abspath('./utils'))
from subprocess import Popen, PIPE
p = Popen(['./utils/summarize_graph', '--in_graph=./tmp/graph.pb'], stdout=PIPE, stderr=PIPE, stdin=PIPE)
output = p.stdout.read()
size = 0
summary = str(output).strip().split(' ')
while summary[summary.index('const') + 1] != 'parameters,':
    starter = summary.index('const')+1
    summary = summary[starter:]
if len(summary) is not 0:
    size = int(summary[summary.index('const') - 2])


In [30]:
size

544

### Test size

In [3]:
import tensorflow as tf
g = tf.Graph()
run_meta = tf.RunMetadata()
# ***** (1) Create Graph *****
g = tf.Graph()
sess = tf.Session(graph=g)
with g.as_default():
    A = tf.Variable(initial_value=tf.random_normal([25, 16]))
    B = tf.Variable(initial_value=tf.random_normal([16, 9]))
    C = tf.matmul(A, B, name='output')
    sess.run(tf.global_variables_initializer())
# *****************************    

In [10]:
import sys, os
from subprocess import Popen, PIPE
import tensorflow as tf
from tensorflow.python.framework import graph_util
def calculate_size(model, sess):
    output_graph_def = graph_util.convert_variables_to_constants(sess, g.as_graph_def(), ['output'])
    try:
        os.mkdir('./tmp')
    except:
        pass
        # print("Use the ./tmp dir as model frozen buffer.")
    sys.path.insert(0, os.path.abspath('./utils'))
    with tf.gfile.GFile('./tmp/graph.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())
    p = Popen(['./utils/summarize_graph', '--in_graph=./tmp/graph.pb'], stdout=PIPE, stderr=PIPE, stdin=PIPE)
    output = p.stdout.read()
    size = 0
    summary = str(output).strip().split(' ')
    while summary[summary.index('const') + 1] != 'parameters,':
        starter = summary.index('const')+1
        summary = summary[starter:]
    if len(summary) > 1:
        size = int(summary[summary.index('const') - 2])
    else:
        raise ValueError(str(output))
    return size

In [11]:
calculate_size(g, sess)

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


544

# Architecture
ref: <https://blog.metaflow.fr/tensorflow-how-to-freeze-a-model-and-serve-it-with-a-python-api-d4f3596b3adc>